In [1]:
from audio_preprocessing.cut_audio_segment_from_audio_file import AudioProcessor
from feature_extraction.feature_extractor import FeatureExtractor
from feature_extraction.mfcc_images_extractor import MelSpectrogramImageExtractor
from data_processing.english_data_processing import ModelEvaluator
from itertools import combinations
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

In [2]:
def extract_mfcc_images(n_mfcc, figsize, input_folder, output_folder):
    mfccImageExtractor = MelSpectrogramImageExtractor(n_mfcc, figsize)
    mfccImageExtractor.extract_from_folders(input_folder, output_folder)

In [3]:
def cut_audio_segment_from_audio_file(frame_size, hop_size, threshold, input_folder, output_folder):
    segmentation = AudioProcessor(frame_size, hop_size, threshold)
    segmentation.process_folders(input_folder, output_folder)

In [4]:
def extract_feature(n_mfcc, input_folder, output_folder, selected_features):
    featureExtractor = FeatureExtractor(n_mfcc)
    featureExtractor.process_folder(input_folder, output_folder, selected_features)
    # featureExtractor.process_folder(input_folder, output_folder, ['mfcc','mfcc_statistics', 'zcr', 'pitch', 'rms'])

In [5]:
def model(featured_data):
    models = [RandomForestClassifier, ExtraTreesClassifier]
    
    df_train_test = pd.read_csv(featured_data)
    df_shuffled = df_train_test.sample(frac=1, random_state=42).reset_index(drop=True)

    modelEvaluator = ModelEvaluator(models)
    best_model, best_scaler, best_avg_acc = modelEvaluator.evaluate_models(df_shuffled)
    return best_model, best_scaler, best_avg_acc

In [6]:
def process_data(frame_size, hop_size, threshold, raw_folder, cutted_folder, n_mfcc_list, featured_data_path, n_mfcc_for_images, images_folder, features):
    cut_audio_segment_from_audio_file(frame_size, hop_size, threshold, raw_folder, cutted_folder)
    n = len(features)

    results = []

    for n_mfcc in n_mfcc_list:
        for r in range(0, n + 1):
            for combo in combinations(range(n), r):
                selected_features = [features[i] for i in range(n) if i not in combo]
                extract_feature(n_mfcc, cutted_folder, featured_data_path, selected_features)
                current_model, current_scaler, current_avg_acc = model(featured_data_path)
                print(f"Scaler: {current_scaler}, Model: {current_model}, n_mfcc: {n_mfcc}, Selected features: {selected_features}, Accuracy: {current_avg_acc}")
                results.append({'Scaler': current_scaler, 'Model': current_model, 'N_MFCC': n_mfcc, 'Selected Features': selected_features, 'Average Accuracy': current_avg_acc})

    results_df = pd.DataFrame(results)
    ranked_results = results_df.sort_values(by='Average Accuracy', ascending=False)

    return ranked_results

In [7]:
process_data(256, 128, 0.015, 'D:/data_analysis/speech_emotion_recognition/data/EnglishDataset/combined_data', 'D:/data_analysis/speech_emotion_recognition/data/EnglishDataset/cleaned_data3', [13,20,26,40], 'D:/data_analysis/speech_emotion_recognition/notebooks/feature4.csv', 100, 'D:/data_analysis/speech_emotion_recognition/data/EnglishDataset/images', ['mfcc','mfcc_statistics', 'zcr', 'pitch', 'rms'])

d:\data_analysis\speech_emotion_recognition\src\audio_preprocessing\utils\audio_utils.py:79: RuntimeWarning: invalid value encountered in divide
  return ste / np.max(ste)
2024-05-22 15:11:13.167 | INFO     | feature_extraction.feature_extractor:process_folder:148 - Processing input folder: D:/data_analysis/speech_emotion_recognition/data/EnglishDataset/cleaned_data3
2024-05-22 15:13:58.017 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature4.csv
2024-05-22 15:13:58.120 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....
2024-05-22 15:15:11.932 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: None, accuracy: 0.6136714781279886
2024-05-22 15:15:28.667 | INFO     | data_processing.english_data_processing:evaluate_models:74

Scaler: MaxAbsScaler, Model: <class 'sklearn.ensemble._forest.RandomForestClassifier'>, n_mfcc: 13, Selected features: ['mfcc', 'mfcc_statistics', 'zcr', 'pitch', 'rms'], Accuracy: 0.6198600814451289


2024-05-22 15:23:42.566 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature4.csv
2024-05-22 15:23:42.650 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....
2024-05-22 15:24:39.230 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: None, accuracy: 0.6107583313858767
2024-05-22 15:24:52.512 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.ExtraTreesClassifier'> - scaler: None, accuracy: 0.618128916667081
2024-05-22 15:25:57.241 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: StandardScaler(), accuracy: 0.6129437922535971
2024-05-22 15:26:11.545 | INFO     | data_

Scaler: None, Model: <class 'sklearn.ensemble._forest.ExtraTreesClassifier'>, n_mfcc: 13, Selected features: ['mfcc_statistics', 'zcr', 'pitch', 'rms'], Accuracy: 0.618128916667081


2024-05-22 15:32:34.102 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature4.csv
2024-05-22 15:32:34.134 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....
2024-05-22 15:33:02.279 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: None, accuracy: 0.5910089649308612
2024-05-22 15:33:11.027 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.ExtraTreesClassifier'> - scaler: None, accuracy: 0.5961060808716652
2024-05-22 15:33:39.622 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: StandardScaler(), accuracy: 0.5911911971638399
2024-05-22 15:33:48.417 | INFO     | data

...

Scaler: StandardScaler, Model: <class 'sklearn.ensemble._forest.RandomForestClassifier'>, n_mfcc: 13, Selected features: ['mfcc_statistics'], Accuracy: 0.6090293212408697


2024-05-22 18:26:28.376 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature4.csv
2024-05-22 18:26:28.398 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....
2024-05-22 18:26:51.659 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: None, accuracy: 0.5601576031199086
2024-05-22 18:26:59.669 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.ExtraTreesClassifier'> - scaler: None, accuracy: 0.566891825819465
2024-05-22 18:27:22.981 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: StandardScaler(), accuracy: 0.5612484275322325
2024-05-22 18:27:30.955 | INFO     | data_

Scaler: RobustScaler, Model: <class 'sklearn.ensemble._forest.ExtraTreesClassifier'>, n_mfcc: 13, Selected features: ['mfcc'], Accuracy: 0.5694402594841146


2024-05-22 18:29:39.631 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature4.csv
2024-05-22 18:29:39.642 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....


ValueError: at least one array or dtype is required